# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re 
import pickle


import nltk 
nltk.download(['punkt', 'punkt_tab', 'wordnet']) 

from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate, RandomizedSearchCV

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\caity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\caity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\caity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponseProject.db')
df = pd.read_sql('SELECT * FROM MsgCat',engine)
X = df['message']
y = df.iloc[:, 4:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

df.head()



,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", ' ', text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)

    return clean_tokens


In [22]:
for message in X[:5]:
    print(message)
    print(tokenize(message))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']
Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']
Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
pipeline = Pipeline([ 
        ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)), 
        ('tfidf', TfidfTransformer()), 
        ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(n_jobs = -1))) 
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
def main(): 
    X = df['message']
    y = df.iloc[:, 4:39]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42) 

    pipeline = Pipeline([ 
        ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)), 
        ('tfidf', TfidfTransformer()), 
        ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(n_jobs = -1))) 
    ]) 

    # train classifier 
    pipeline.fit(X_train, y_train)

    # predict on test data 
    y_pred = pipeline.predict(X_test) 


main()

KeyboardInterrupt: 

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

In [67]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x000001CFCC4594E0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [71]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [126]:
cat_list = list(df.iloc[:, 4:])
for category in range(len(cat_list)-1):
    print(cat_list[category], 'classification')
    print(classification_report(y_test.iloc[:, category].values, y_pred[:, category]))
  

related classification
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1873
           1       0.75      1.00      0.86      5934
           2       0.00      0.00      0.00        58

    accuracy                           0.75      7865
   macro avg       0.25      0.33      0.29      7865
weighted avg       0.57      0.75      0.65      7865

request classification
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      6533
           1       1.00      0.00      0.00      1332

    accuracy                           0.83      7865
   macro avg       0.92      0.50      0.45      7865
weighted avg       0.86      0.83      0.75      7865

offer classification
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macr

C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packag

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7787
           1       0.00      0.00      0.00        78

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.98      0.99      0.99      7865

shops classification
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7837
           1       0.00      0.00      0.00        28

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865

aid_centers classification
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7762
           1       0.00      0.00      0.00       103

    accuracy                           0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97   

C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packag

### 6. Improve your model
Use grid search to find better parameters. 

In [101]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x000001CFCC4594E0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x000001CFCC4594E0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfi

In [108]:
parameters = {
    'clf__estimator__max_depth': [10], 
    'clf__estimator__min_samples_leaf': [2, 5, 10],
    'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, parameters)




In [110]:
model = cv.fit(X_train, y_train)

In [99]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__monotonic_cst', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__esti

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [112]:
cv.best_params_


{'clf__estimator__max_depth': 10,
 'clf__estimator__min_samples_leaf': 10,
 'clf__estimator__n_estimators': 20}

In [128]:

y_pred= model.predict(X_test)


for category in range(len(cat_list)-1):
    print(cat_list[category], 'classification')
    print(classification_report(y_test.iloc[:, category].values, y_pred[:, category]))

related classification
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1873
           1       0.75      1.00      0.86      5934
           2       0.00      0.00      0.00        58

    accuracy                           0.75      7865
   macro avg       0.25      0.33      0.29      7865
weighted avg       0.57      0.75      0.65      7865

request classification
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      6533
           1       1.00      0.00      0.00      1332

    accuracy                           0.83      7865
   macro avg       0.92      0.50      0.45      7865
weighted avg       0.86      0.83      0.75      7865

offer classification
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macr

C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packag

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7473
           1       0.00      0.00      0.00       392

    accuracy                           0.95      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.90      0.95      0.93      7865

electricity classification
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7697
           1       0.00      0.00      0.00       168

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.96      0.98      0.97      7865

tools classification
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7817
           1       0.00      0.00      0.00        48

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99   

C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\caity\anaconda3\Lib\site-packag

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [130]:
parameters = {
    'clf__estimator__max_depth': [10], 
    'clf__estimator__min_samples_leaf': [10],
    'clf__estimator__n_estimators': [20]
}

cv = GridSearchCV(pipeline, parameters)

model2 = cv.fit(X_train, y_train)

### 9. Export your model as a pickle file

In [132]:
filename = 'ML_Pipeline_Model.sav'
pickle.dump(model2, open(filename, 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.